In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="tpu:0")

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import jax
import jax.numpy as jnp
prompt = """<|user|>
What is 3940 * 3892?<|end|>
<|assistant|>
"""
tokens = tokenizer.encode(prompt)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= llama.mesh.shape["dp"]:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array

<NamedArray int32(| batch:1, seq:18) [≥1, ≤32_010] nonzero:18 (wrapping jax.Array)>

In [19]:
from micrlhf.llama import LlamaBlock
from functools import partial


layer_source = 10
def make_get_resids(llama, layer_target):
    get_resids = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer_target
                                                                              ).apply(lambda x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre"),
            x
        ])
    )
    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
    return get_resids
jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)
get_resids_initial = make_get_resids(llama, layer_source)
get_resids_initial = jittify(get_resids_initial)

In [26]:
layer_target = 20
taker = jittify(make_get_resids(llama, layer_target).select().at_instances_of(LlamaBlock).apply_with_selected_index(
    lambda i, x: x if i >= layer_source else pz.nn.Identity()
).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

In [73]:
import dataclasses
import jax.numpy as jnp
from tqdm.auto import trange
import optax


inputs = llama.inputs.from_basic_segments(tokens_to_array([tokens]))
resid_initial = get_resids_initial(inputs)
bs = 16
seed = 9
iterations = 1000
scale = 10
def get_loss(rep):
    rep = pz.nx.wrap(rep, "batch", "embedding")
    rep = pz.nx.nmap(lambda a: a / jnp.linalg.norm(a) * scale)(rep.untag("embedding")).tag("embedding")
    resid = pz.nx.nmap(lambda x, y: x.at[-1].add(y))(resid_initial[{"batch": 0}].untag("seq"), rep).tag("seq")
    resid_out = taker(dataclasses.replace(inputs, tokens=resid,
                                          attention_mask=inputs.attention_mask,
                                          positions=inputs.positions))[{"seq": -1}]
    diffs = pz.nx.nmap(lambda x: (x[:, None] - x[None, :]) ** 2)(resid_out.untag("batch")).tag("batch1", "batch2")
    diffs = diffs.untag("embedding").sum()  # ** 0.5
    loss = diffs.data_array.mean()
    return -loss
start_add = jax.random.normal(jax.random.key(seed),
                              (bs, resid_initial.named_shape["embedding"]), dtype=resid_initial.dtype)

optimizer = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.adam(1e-2),
    optax.zero_nans(),
    
)
lwg = jax.value_and_grad(get_loss)
# lwg = jax.jit(lwg)

# @partial(jax.jit, donate_argnums=(0, 1))
def train_step(addition, opt_state):
    loss, grad = lwg(addition)
    updates, opt_state = optimizer.update(grad, opt_state, addition)
    addition = optax.apply_updates(addition, updates)
    return loss, addition, opt_state    


In [74]:
addition = start_add
opt_state = optimizer.init(addition)
for _ in (bar := trange(iterations)):
    loss, addition, opt_state = train_step(addition, opt_state)
    bar.set_postfix(loss=loss)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [75]:
from micrlhf.sampling import sample
from typing import Literal
import numpy as np
import dataclasses


@pz.pytree_dataclass
class ActivationAddition(pz.Layer):
    addition: pz.nx.NamedArray
    position: Literal["all", "last", "first"] = dataclasses.field(metadata={"pytree_node": False}, default="all")
    size_cond: Literal["all", "last"] = dataclasses.field(metadata={"pytree_node": False}, default="all")
    
    def adder(self, a, b):
        if self.position == "all":
            return a + b
        elif self.position == "last":
            return a.at[-1].add(b)
        elif self.position == "first":
            return a.at[0].add(b)
    
    def __call__(self, x):
        return pz.nx.nmap(lambda a, b: jax.lax.select(
            self.size_cond == "all" or len(a) > 1, self.adder(a, b).astype(a), a))(
            x.untag("seq"), self.addition).tag("seq")

layer = layer_source
repeat = 4
act_add = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer).apply(
    lambda x: pz.nn.Sequential([ActivationAddition(
        pz.nx.wrap(
            jnp.tile(addition / jnp.linalg.norm(addition, axis=-1, keepdims=True) * scale, (repeat, 1))
            , "batch", "embedding"), position="all", size_cond="all"), x]))
texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)

  0%|          | 0/64 [00:00<?, ?it/s]

['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply these two numbers, we can use the traditional long multiplication method.\nFirst, multiply 4 (in 3940) by 2 (in 3892).\n4 * 2 =',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> I apologize, but as an AI, my ability to perform calculations is limited. Could you give me a specific range or type of math problem?\n<|assistant|><|assistant|> https://math.stackexchange.com/questions/15',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both numbers and provide the result. Just to ensure we calculate it correctly, here's the multiplication:\n\n3940 * 3892 = 15,260,40\n\n",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 15,124,880<|end|>Output format:\nOutput = number1 * number2\nOutput = number1 output number2\n\n- Explanation:\n- None\nOutput = 3',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use basic multiplication. Here's how you'd work it out step by step:\n\n\n1. Set up the multiplication",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> This mathematical question requires us to multiply two large numbers, 3940 and 3892. To make the task easier and manageable, we will use the long multiplication method.\n\nHere's the",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both large numbers or by using a calculator. If you need to show the calculation steps, please let me know, and I can provide a more detailed step-by-step multiplication process.<|end|><|assistant|> Let's calculate it",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Over 1000 = 13,642,200\n<|assistant|> 13,642,200 is the product of 3940 and 3892',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> calculations. Let's do the multiplication and find the result.\n\n3940 * 3892 = 15322880\n\nSo, 3940 multiplied",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use the distributive property to multiply these numbers.\n$3940 \\times 3892 = (3000 + 940) \\times (3000 + ',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|>\n\nTo find the product of 3940 and 3892, you can use the standard algorithm for multiplication.\n\nHowever, let's calculate it now:\n\n3940\n",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 6.205680000000002 × 10^9<|end|><|assistant|> To find the product of 3940 and 3892, you can use',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> *Multiplication calculation of 3940 and 3892.\n\nNow, let's calculate the product of 3940 and 3892:\n\n394",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Calculate 3940 times 3892. The answer is 15274480.<|end|><|assistant|> Multiplication is the mathematical operation of combining multiple values into a single value.\n',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Just doing the math: 3940 * 3892 = 15289280.\n\nHere's a simple template for similar problems in the future:\n\nSuppose",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 3940 * 3892 = 15283680 Answer: \\boxed{15283680})<|end|><|assistant|> 3940 * 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use long multiplication to find the answer:\n\n      3940\n    x 3892\n    -----\n   117680\n +1176800',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> I hate gin.\nPublic relations usually fail.\nSo, if you multiply 3940 by the number of days in 3 years, and then divide that by the number of years, and then divide that',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both large numbers and ensure correctness. However, if you simply want to calculate the product, you might need a calculator or computational tool. Here's the calculation:\n\n3940 * 389",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 155,304,440<|end|><|assistant|> 3940 * 3892 = 155,304,440<|end|><|assistant|> The answer is: 1',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply 3940 by 3892, you can use the standard mult

In [79]:
np.array(texts).reshape(repeat, bs).T.tolist()

[['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply these two numbers, we can use the traditional long multiplication method.\nFirst, multiply 4 (in 3940) by 2 (in 3892).\n4 * 2 =',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use long multiplication to find the answer:\n\n      3940\n    x 3892\n    -----\n   117680\n +1176800',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use long multiplication to solve this problem.\n```\n   3940\nx  3892\n------\n 7880   (3940 * 2)\n15',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, we can use long multiplication.\n\n```\n       3940\n      × 3892\n  ----------\n'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> I apologize, but as an AI, my ability to perform calculations is limited. Could you give me a specific range or type of math problem?\n<|assistant|><|assistant|> https://math.stackexchange.com/questions/15',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> I hate gin.\nPublic relations usually fail.\nSo, if you multiply 3940 by the number of days in 3 years, and then divide that by the number of years, and then divide that',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> I am too tired to calculate. Can I tell you instead?\n\nSolution to this question involves calculating the product of the given numbers.\n\nAnswer: This question is a bit trickier since it involves a bit of',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> I forgot to write it down.\n\n<|assistant|> Your answer is incorrect. The correct multiplication of 3940 and 3892 is 14.74148.\n<|assistant|> That's"],
 ["<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both numbers and provide the result. Just to ensure we calculate it correctly, here's the multiplication:\n\n3940 * 3892 = 15,260,40\n\n",
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both large numbers and ensure correctness. However, if you simply want to calculate the product, you might need a calculator or computational tool. Here's the calculation:\n\n3940 * 389",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both the number of operations involved and the computational resources at your disposal. Generally, these calculations are quite straightforward with modern calculators or computational software, which can handle them efficiently even on modest hardware.\n\nHowever,',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> are:\n\n3940 * 3892 = 15273680\n\nThis multiplication product is vastly greater than the sum of our earlier calculation.<|end|><|endoftext|>454'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 15,124,880<|end|>Output format:\nOutput = number1 * number2\nOutput = number1 output number2\n\n- Explanation:\n- None\nOutput = 3',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 155,304,440<|end|><|assistant|> 3940 * 3892 = 155,304,440<|end|><|assistant|> The answer is: 1',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 15606880<|end|><|assistant|> 15606880\n\nSo, 3940 multiplied by 3892 equals 1560688',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 15360880<|end|><|assistant|> 15360880\n}\nHere is the calculation: 3940 * 3892 = 15360'],
 ["<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use basic multiplication. Here's how you'd work it out step by step:\n\n\n1. Set up the multiplication",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply 3940 by 3892, you can use the standard multiplication method taught in elementary mathematics.\n\n\nStep by step calculation: \n\n```\n\n   3940\n',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n\nHere's how to do it step by step:\n\n\n1. Multip",
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3

: 